# Q3: Who are most likely to cause traffic accidents?

**Objective：Analysing the impact of driver and vehicle? characteristics on crash severity and identifying high-risk features**

* Dependent Variables：accident_severity(from collision) 
* Independent Variables：
    - driver features：sex_of_driver、age_of_driver、driver_imd_decile
    - vehicle features：vehicle_type、vehicle_reference。
* mian dataset: dft-road-casualty-statistics-vehicle-2023.csv
* other dataset: dft-road-casualty-statistics-collision-2023.csv

In [2]:
import pandas as pd
import numpy as np

import statsmodels
import statsmodels.api as sm

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sn

In [3]:
dtype_dict = {0: 'str', 2: 'str'}  # columns 0 and 2 are set to strings
vehicle_data = pd.read_csv('data/dft-road-casualty-statistics-vehicle-2023.csv', dtype=dtype_dict)#, low_memory=False
collision_data = pd.read_csv('data/dft-road-casualty-statistics-collision-2023.csv', dtype=dtype_dict)#, low_memory=False)
#vehicle_data.info()
#collision_data.info()

In [22]:
#vehicle_data
#collision_data

,accident_index,accident_year,accident_reference,vehicle_reference,vehicle_type,towing_and_articulation,vehicle_manoeuvre,vehicle_direction_from,vehicle_direction_to,vehicle_location_restricted_lane,...,generic_make_model,driver_imd_decile,driver_home_area_type,lsoa_of_driver,escooter_flag,dir_from_e,dir_from_n,dir_to_e,dir_to_n,driver_distance_banding
0,2.02301E+12,2023,10419171,1,11,0,4,1,5,0,...,ALEXANDER DENNIS MODEL MISSING,3,1,E01001177,0,NaN,NaN,NaN,NaN,2
1,2.02301E+12,2023,10419183,1,11,0,18,5,1,0,...,WRIGHTBUS GEMINI,6,1,E01001419,0,NaN,NaN,NaN,NaN,2
2,2.02301E+12,2023,10419183,2,9,0,9,1,6,0,...,TOYOTA YARIS,3,1,E01001546,0,NaN,NaN,NaN,NaN,1
3,2.02301E+12,2023,10419183,3,9,0,8,7,1,0,...,BMW 2 SERIES,4,1,E01001686,0,NaN,NaN,NaN,NaN,4
4,2.02301E+12,2023,10419189,1,9,0,18,7,3,0,...,LEXUS RX 400,5,1,E01002443,0,NaN,NaN,NaN,NaN,1


In [4]:
# Select the columns of interest for analysis
columns_of_interest1 = [
    'accident_index', 'vehicle_reference', 
    'vehicle_type', 'sex_of_driver', 'age_of_driver', 
    'age_band_of_driver', 'driver_imd_decile'
]
columns_of_interest2 = [
    'accident_index', 'accident_severity'
]
vehicle_data_selected = vehicle_data[columns_of_interest1]
collision_data_selected = collision_data[columns_of_interest2]

# Display the summary of the selected columns to check their data types and missing values
vehicle_data_selected.info()
collision_data_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189815 entries, 0 to 189814
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   accident_index      189815 non-null  object
 1   vehicle_reference   189815 non-null  int64 
 2   vehicle_type        189815 non-null  int64 
 3   sex_of_driver       189815 non-null  int64 
 4   age_of_driver       189815 non-null  int64 
 5   age_band_of_driver  189815 non-null  int64 
 6   driver_imd_decile   189815 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 10.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104258 entries, 0 to 104257
Data columns (total 2 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   accident_index     104258 non-null  object
 1   accident_severity  104258 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.6+ MB


In [ ]:
# merge data through accident_index
merged_data = vehicle_data_selected.merge(
    collision_data_selected,
    on='accident_index',
    how='left'
)
merged_data.head()

In [ ]:
# delete accident_index, because it is unnecessary now
